In [16]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

# Load the preprocessed data
modelling_table = pd.read_csv("preprocessed_data.csv")

# valid_datetimeをdatetime型に変換
modelling_table['valid_datetime'] = pd.to_datetime(modelling_table['valid_datetime'])

# 季節ごとのデータフレームを作成
modelling_table['month'] = modelling_table['valid_datetime'].dt.month
spring = modelling_table[modelling_table['month'].isin([3, 4, 5])]
summer = modelling_table[modelling_table['month'].isin([6, 7, 8])]
autumn = modelling_table[modelling_table['month'].isin([9, 10, 11])]
winter = modelling_table[modelling_table['month'].isin([12, 1, 2])]

# 季節ごとの総発電量モデル
total_models = {
    'spring': smf.quantreg('total_generation_MWh ~ bs(Radiation_dwd,df=5) + bs(Radiation_ncep,df=5) + bs(WindSpeed_dwd,df=8) + bs(WindSpeed_ncep,df=8)', data=spring),
    'summer': smf.quantreg('total_generation_MWh ~ bs(Radiation_dwd,df=5) + bs(Radiation_ncep,df=5) + bs(WindSpeed_dwd,df=8) + bs(WindSpeed_ncep,df=8)', data=summer),
    'autumn': smf.quantreg('total_generation_MWh ~ bs(Radiation_dwd,df=5) + bs(Radiation_ncep,df=5) + bs(WindSpeed_dwd,df=8) + bs(WindSpeed_ncep,df=8)', data=autumn),
    'winter': smf.quantreg('total_generation_MWh ~ bs(Radiation_dwd,df=5) + bs(Radiation_ncep,df=5) + bs(WindSpeed_dwd,df=8) + bs(WindSpeed_ncep,df=8)', data=winter)
}

forecast_models = dict()

# 季節ごとの太陽光発電と風力発電の分位数を計算
for quantile in range(10, 100, 10):
    for season, season_months in {'spring': [3, 4, 5], 'summer': [6, 7, 8], 'autumn': [9, 10, 11], 'winter': [12, 1, 2]}.items():
        forecast_models[f"{season}_q{quantile}"] = total_models[season].fit(q=quantile/100, max_iter=2500)
        modelling_table.loc[modelling_table['month'].isin(season_months), f"{season}_q{quantile}"] = forecast_models[f"{season}_q{quantile}"].predict(modelling_table.loc[modelling_table['month'].isin(season_months)])

# 各季節ごとの予測結果を保存
modelling_table.to_csv("analyzed_data_sea.csv", index=False)

seasons = ['spring', 'summer', 'autumn', 'winter']
# 全ての量子回帰モデルを"models_sea"ファイルに保存
for season in seasons:
    for quantile in range(10, 100, 10):
        forecast_models[f"{season}_q{quantile}"].save(f"models_sea/model_{season}_q{quantile}.pickle")

/usr/local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (2500) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/usr/local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (2500) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/usr/local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (2500) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


In [14]:
spring

,ref_datetime,valid_datetime,WindSpeed_dwd,WindSpeed_ncep,Radiation_dwd,Radiation_ncep,Wind_MWh_credit,Solar_MWh_credit,total_generation_MWh,month
62512,2021-02-27 00:00:00+00:00,2021-03-01 00:00:00+00:00,2.413352,2.991264,-0.017334,0.0,1.413,0.0,1.413,3
62513,2021-02-27 06:00:00+00:00,2021-03-01 00:00:00+00:00,1.183161,4.129822,-0.005748,0.0,1.413,0.0,1.413,3
62514,2021-02-27 12:00:00+00:00,2021-03-01 00:00:00+00:00,1.150469,4.588514,0.002937,0.0,1.413,0.0,1.413,3
62515,2021-02-27 18:00:00+00:00,2021-03-01 00:00:00+00:00,2.003273,4.629436,0.003778,0.0,1.413,0.0,1.413,3
62516,2021-02-28 00:00:00+00:00,2021-03-01 00:00:00+00:00,1.428826,5.059417,0.006799,0.0,1.413,0.0,1.413,3
...,...,...,...,...,...,...,...,...,...,...
380807,2023-05-30 18:00:00+00:00,2023-05-31 23:30:00+00:00,7.871054,8.099297,-0.006633,0.0,125.111,0.0,125.111,5
380808,2023-05-31 00:00:00+00:00,2023-05-31 23:30:00+00:00,8.005945,6.825882,-0.005801,0.0,125.111,0.0,125.111,5
380809,2023-05-31 06:00:00+00:00,2023-05-31 23:30:00+00:00,7.851712,6.999551,-0.009692,0.0,125.111,0.0,125.111,5
380810,2023-05-31 12:00:00+00:00,2023-05-31 23:30:00+00:00,7.743046,7.333244,0.002806,0.0,125.111,0.0,125.111,5
